In [66]:
!pip install datasets transformers nltk evaluate tqdm

In [67]:
from tqdm import tqdm
import sys
import os
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import corpus_bleu
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("blue_score.ipynb"), "..")))
from datasets import load_dataset

# from transformers_models.marian.marianMT import


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:

def retrieve_data():
    dataset = load_dataset("wmt14", "de-en")

    subset = dataset["train"].select(range(120000))

    # train_data = subset.select(range(100000))
    # val_data = subset.select(range(100000, 110000))
    # test_data = subset.select(range(110000, 120000))



    train_data = subset.select(range(15000))
    val_data = subset.select(range(15000, 16000))
    test_data = subset.select(range(16000, 17000))
    return {
        "train": train_data,
        "validation": val_data,
        "test": test_data
    }


In [69]:
def translated(n, model):
    return model.translate_text(n)

In [70]:

data = retrieve_data()


In [71]:
data["validation"]["translation"]

[{'de': 'Der Vorschlag für eine Verordnung verfolgt meines Erachtens das Ziel, die Definitionen und die Bewertungen einiger Wirtschaftsvorgänge zu aktualisieren, die bei der Berechnung des Defizits zu berücksichtigen sind, wie im Vertrag von Maastricht dargelegt, in dem die maximalen Prozentsätze festgelegt werden, die das Defizit der öffentlichen Verwaltungen im Verhältnis zum BIP zu Marktpreisen in den einzelnen Staaten umfassen darf.',
  'en': 'In our view, the main purpose of the proposal for a regulation is to update the definitions and the assessments of some of the economic operations involved in the calculation of deficits, as reflected in the Treaty establishing the European Community, the Treaty of Maastricht, which lays down the maximum percentages which the government deficit must not exceed as a ratio of GDP at market prices in the various Member States.'},
 {'de': 'Bisher berechneten die Staaten dieses BIP gemäß der Methodik des ESVG 79. Die letzte Veranlagung, die uns vo

In [72]:
data["test"]["translation"]

[{'de': 'Die Prodi-Kommission möchte jetzt aber die Europäische Union verändern, um mit den Vereinigten Staaten, mit denen sie gleichzeitig konkurriert und rivalisiert, einen gemeinsamen Führungsanspruch auf der Weltbühne zu erheben.',
  'en': 'But the Prodi Commission now wants to transform the EU so that it can apply for joint global governance with the United States, which it also sees as its future competitor and rival.'},
 {'de': 'Diese neue imperiale Politik beinhaltet eine Reform der europäischen und der einzelstaatlichen Institutionen, die Stärkung staatlicher Macht, einen schärferen Wettbewerb, eine europäische Armee sowie die Neuverteilung der Finanzmittel.',
  'en': 'This new imperial policy advocates reorganising European and national institutions, strengthening the power of the state, increasing competition, establishing a European army and redistributing financial resources.'},
 {'de': 'Daß man noch wagt, von der Verteidigung eines "europäischen Gesellschaftsmodells " zu 

In [73]:
data["train"]["translation"]

[{'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'},
 {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
  'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'},
 {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
  'en': "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."},
 {'de': 'Im Parlament besteht der Wunsch nach einer Aussprache im

In [74]:
from transformers import MarianMTModel, MarianTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [75]:
english = [n["en"] for n in data["train"]["translation"]]
german = [n["de"] for n in data["train"]["translation"]]

In [76]:
# translation_pairs_en2de = {"en":[], "de":[]}


# for pair in data["train"]["translation"]:
#     translation_pairs_en2de["en"].append(pair["en"])
#     translation_pairs_en2de["de"].append(pair["de"])

In [77]:
# translation_pairs_en2de

In [78]:
# translation_pairs_en2de_input_output = [
#     {"en": src, "de": tgt}
#     for src, tgt in zip(translation_pairs_en2de["en"], translation_pairs_en2de["de"])
# ]

In [79]:
data

{'train': Dataset({
     features: ['translation'],
     num_rows: 15000
 }),
 'validation': Dataset({
     features: ['translation'],
     num_rows: 1000
 }),
 'test': Dataset({
     features: ['translation'],
     num_rows: 1000
 })}

In [80]:
def preprocess(batch):
    inputs = tokenizer(
        batch["translation"]["en"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(
            batch["translation"]["de"],
            truncation=True,
            padding="max_length",
            max_length=128
        )
    inputs["labels"] = targets["input_ids"]
    return inputs


In [81]:
tokenized_data = {
    "train": data["train"].map(preprocess),
    "validation": data["validation"].map(preprocess)
}


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [94]:
from tqdm import tqdm
import evaluate

bleu = evaluate.load("bleu")
test_subset = data["test"].select(range(1000))

src_texts = [ex["translation"]["en"] for ex in test_subset]
references = [[ex["translation"]["de"]] for ex in test_subset]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
batch_size = 16
predictions = []

for i in tqdm(range(0, len(src_texts), batch_size), desc="Translating"):
    batch = src_texts[i:i + batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length = 128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, num_beams=4, max_length = 128, early_stopping=True)
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(preds)


Translating: 100%|██████████| 63/63 [00:47<00:00,  1.32it/s]


In [95]:
predictions

['Aber jetzt will die Prodi-Kommission die EU so verändern, dass sie eine gemeinsame globale Governance mit den Vereinigten Staaten beantragen kann, die sie auch als ihren künftigen Konkurrenten und Rivalen sieht.',
 'Diese neue kaiserliche Politik setzt sich dafür ein, die europäischen und nationalen Institutionen neu zu organisieren, die Macht des Staates zu stärken, den Wettbewerb zu verstärken, eine europäische Armee aufzubauen und die finanziellen Ressourcen neu zu verteilen.',
 'Auch wenn man es wagt, auf die Verteidigung eines "europäischen Sozialmodells" zu verweisen, ist es eine beschämende Lüge, denn je mehr die EU die Vereinigten Staaten bekämpft, desto mehr wird sie zu ihrem Klon.',
 'Wir werden gegen diese Ziele stimmen, weil wir sowohl im Parlament als auch auf der Straße gegen diese Politik kämpfen.',
 'Eine europäische Charta der Grundrechte wäre der einzige Lichtstrahl in dieser Politik, aber wieder einmal gibt es keine Bestimmung, die die Menschen in Europa selbst daz

In [92]:
print(references)

[['Die Prodi-Kommission möchte jetzt aber die Europäische Union verändern, um mit den Vereinigten Staaten, mit denen sie gleichzeitig konkurriert und rivalisiert, einen gemeinsamen Führungsanspruch auf der Weltbühne zu erheben.'], ['Diese neue imperiale Politik beinhaltet eine Reform der europäischen und der einzelstaatlichen Institutionen, die Stärkung staatlicher Macht, einen schärferen Wettbewerb, eine europäische Armee sowie die Neuverteilung der Finanzmittel.'], ['Daß man noch wagt, von der Verteidigung eines "europäischen Gesellschaftsmodells " zu sprechen, ist eine schändliche Lüge, denn je mehr sich die Europäische Union gegen die Vereinigten Staaten stellt, desto mehr wird sie zu ihrem getreuen Abbild.'], ['Wir stimmen gegen diese Ziele, da wir eine solche Politik im Parlament und auf der Straße bekämpfen.'], ['Die Ausarbeitung einer europäischen Charta der Grundrechte wäre der einzige kleine Hoffnungsschimmer in dieser Politik, doch ist wieder nicht die Möglichkeit vorgesehen

In [ ]:
!pip install bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [96]:
results = bleu.compute(predictions=predictions, references=references)
print(f"BLEU before fine-tunning score: {results['bleu']:.4f}")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

print("METEOR:", meteor.compute(predictions=predictions, references=[r[0] for r in references]))
print("BERTScore:", bertscore.compute(predictions=predictions, references=[r[0] for r in references], lang="de"))



BLEU before fine-tunning score: 0.2760


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


ImportError: To be able to use evaluate-metric/bertscore, you need to install the following dependencies['bert_score'] using 'pip install bert_score' for instance'

In [86]:
results

{'bleu': 0.06728764356452276,
 'precisions': [0.14645694951644286,
  0.07988890071711297,
  0.05098989583129752,
  0.03436059879412258],
 'brevity_penalty': 1.0,
 'length_ratio': 3.9613851235903863,
 'translation_length': 104331,
 'reference_length': 26337}

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58101, 512, padding_idx=58100)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58101, 512, padding_idx=58100)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [88]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./marianmt-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=7,
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

trainer.train()


In [ ]:
import matplotlib.pyplot as plt

log_history = trainer.state.log_history
steps = []
losses = []

for entry in log_history:
    if "loss" in entry and "epoch" in entry:
        steps.append(entry["epoch"])
        losses.append(entry["loss"])

plt.figure(figsize=(8, 5))
plt.plot(steps, losses, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss per Epoch")
plt.grid(True)
plt.show()


In [ ]:
bleu = evaluate.load("bleu")
test_subset = data["test"].select(range(1000))

src_texts = [ex["translation"]["en"] for ex in test_subset]
references = [[ex["translation"]["de"]] for ex in test_subset]

batch_size = 16
predictions = []

for i in tqdm(range(0, len(src_texts), batch_size), desc="Translating"):
    batch = src_texts[i:i + batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(preds)

In [ ]:
results = bleu.compute(predictions=predictions, references=references)
print(f"BLEU score after fine-tuning: {results['bleu']:.4f}")

In [ ]:
model.save_pretrained("marianmt-finetuned")
tokenizer.save_pretrained("marianmt-finetuned")
